In [1]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import glob
import os

# Keras

In [2]:
from header.Keras.Wavenet_model import Wavenet
from header.Keras.dataset import generate_data
from header.Keras.train_and_test import scheduler, train_val_plot, rolling_testing, testing
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.random import uniform
import tensorflow as tf
tf.keras.backend.set_floatx('float64')

# Read data

In [3]:
# Read data
lag = 44
gap = 22
check_period = 5
check_size = 88

df = pd.read_csv('./csv_dataset/data_prepro_0814.csv').dropna(axis = 0).reset_index(drop = True)

model = Wavenet(time_step = 44, feature_num = 103, kernel_size = 5,
                     num_blocks = 3, num_layers = 3, output_channel = 103, reg = 1e-2)
model.load_weights('./Keras_model_ckpt/best_model.ckpt')

# Without rolling

In [ ]:
data_info = {
    'label': 'label', 
    'date': 'tmp_index_ts_value_20200430_henry.tx_dt',
    'start': '2018-01-01',
    'end': None
}

rolling_info = {
    'lag': 44,
    'gap': 22,
    'check_period': 5,
    'check_size': 88,
    'sample': True,
}

acc_history_without_rolling = testing(model = model, model_tag = 'dl', data = df, data_info = data_info,
                                              rolling_info = rolling_info)

mean, std = acc_history_without_rolling.mean(), acc_history_without_rolling.std()
print('mean = {:.2f}, std = {:.2f}'.format(mean, std))

# With rolling

In [4]:
# Some callbacks
metric = 'binary_accuracy'
optimizer = Adam(lr = 1e-4, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, decay = 0.0, amsgrad = False)
Scheduler = LearningRateScheduler(scheduler)
Early_stopping = EarlyStopping(monitor = "val_" + metric, min_delta = 1e-4, patience = 10, 
                               verbose = 1, mode = "auto", baseline = None, 
                               restore_best_weights = True)

data_info = {
    'label': 'label', 
    'date': 'tmp_index_ts_value_20200430_henry.tx_dt',
    'start': '2018-01-01',
    'end': None
}

rolling_info = {
    'lag': 44,
    'gap': 22,
    'check_period': 5,
    'check_size': 88,
    'sample': True,
    'optimizer': optimizer,
    'metric': metric,
    'scheduler': Scheduler,
    'early_stop': Early_stopping
}

ma_result = np.load('ma_result.npy')

acc_history, pred = rolling_testing(model = model, model_tag = 'dl', data = df, data_info = data_info,
                              baseline = ma_result, rolling_info = rolling_info)

mean, std = acc_history.mean(), acc_history.std()
print('mean = {:.2f}, std = {:.2f}'.format(mean, std))

Roll the model from 2018-01-01 to 2020-02-25.
Day 2018-01-05 start to retrain...
Ma, model = 62.07, 34.48
Epoch 1/20
 6/12 [==============>...............] - ETA: 7s - loss: 1.7174 - binary_accuracy: 0.5124

KeyboardInterrupt: 